In [2]:
import os ,json, logging
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool,StructuredTool
load_dotenv(dotenv_path='.env')
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
llm=ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.2
)

In [7]:
@tool
def get_exchange_rate(pair:str)->str:
    """Tra cứu tỷ giá cơ bản (demo). pair ví dụ: 'USD/VND'."""
    rates = {"USD/VND": 24500, "EUR/VND": 26600}
    return str(rates.get(pair.upper(), "Unknown"))


In [8]:
llm_tools=llm.bind_tools([get_exchange_rate])

In [9]:

resp = llm_tools.invoke("Hỏi: Tỷ giá USD/VND hiện tại là bao nhiêu?")
print(type(resp), resp)

<class 'langchain_core.messages.ai.AIMessage'> content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 72, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYWKt9EiyVe0RyCnJVlzFK7hs7U1S', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--a942831e-08e6-49d3-907c-eb86e44048ef-0' tool_calls=[{'name': 'get_exchange_rate', 'args': {'pair': 'USD/VND'}, 'id': 'call_1sER8IBrFcdCchbfAS6rCdAC', 'type': 'tool_call'}] usage_metadata={'input_tokens': 72, 'output_tokens': 17, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [11]:
from langchain_core.messages import ToolMessage

# 1️⃣ Gọi LLM lần đầu
resp = llm_tools.invoke("Hỏi: Tỷ giá USD/VND hiện tại là bao nhiêu?")

print("Tool calls:", resp.tool_calls)
# resp.content rỗng vì LLM mới yêu cầu gọi tool thôi

# 2️⃣ Nếu có tool_calls thì chạy tool thật
if resp.tool_calls:
    call = resp.tool_calls[0]  # ví dụ chỉ 1 tool
    tool_name = call["name"]
    args = call["args"]
    tool_id = call["id"]

    # Gọi tool thật
    result = get_exchange_rate.invoke(args)

    # Gửi lại kết quả cho LLM qua ToolMessage
    follow_up = llm_tools.invoke([
        resp,  # AIMessage trước đó
        ToolMessage(content=str(result), tool_call_id=tool_id)
    ])

    print("✅ Final answer:", follow_up.content)
else:
    print("✅ Final answer:", resp.content)


Tool calls: [{'name': 'get_exchange_rate', 'args': {'pair': 'USD/VND'}, 'id': 'call_Pu7PBJl9Qfp0cIYqSwyGqW2v', 'type': 'tool_call'}]
✅ Final answer: The current exchange rate for USD to VND is 24,500 VND for 1 USD.
